In [ ]:
"""


Real-time ASL demo on a laptop:
- Captures frames from the built-in webcam.
- Extracts a hand region of interest (ROI) and preprocesses to 28x28.
- Runs the full ASL CNN on the CPU.
- Overlays the predicted ASL letter on the video stream and shows the ROI box.

Designed for a short demo video with signs: D, I, E, U, and an "unknown" open-hand.
"""

import cv2
import numpy as np
import tensorflow as tf
from time import perf_counter

# -----------------------------
# CNN and labels
# -----------------------------
IMG_H, IMG_W, IMG_C = 28, 28, 1

# Label mapping (adapt to your training label order)
ASL_CLASSES = [
    'A','B','C','D','E','F','G','H','I','K',
    'L','M','N','O','P','Q','R','S','T','U',
    'V','W','X','Y'
]
UNKNOWN_LABEL = 'UNK'

print("Loading full ASL CNN model...")
model = tf.keras.models.load_model("asl_cnn_full.h5")
print("Model loaded.")
# Optional: model.summary()

# -----------------------------
# Helper: classify one ROI
# -----------------------------

def classify_roi(gray_roi):
    """
    gray_roi: 2D uint8 image (cropped hand), arbitrary size.
    Returns (pred_label, pred_prob, latency_ms).
    """
    # Resize to 28x28 and normalize
    roi_resized = cv2.resize(gray_roi, (IMG_W, IMG_H),
                             interpolation=cv2.INTER_AREA)
    roi_norm = roi_resized.astype(np.float32) / 255.0
    roi_norm = roi_norm.reshape(1, IMG_H, IMG_W, IMG_C)

    t0 = perf_counter()
    logits = model.predict(roi_norm, batch_size=1, verbose=0)[0]  # (25,)
    t1 = perf_counter()

    probs = tf.nn.softmax(logits).numpy()
    pred_idx = int(np.argmax(probs))
    pred_prob = float(probs[pred_idx])

    if pred_prob < 0.6:   # rejection threshold; tune as needed
        pred_label = UNKNOWN_LABEL
    else:
        pred_label = ASL_CLASSES[pred_idx]

    latency_ms = (t1 - t0) * 1000.0
    return pred_label, pred_prob, latency_ms

# -----------------------------
# Demo loop with laptop webcam
# -----------------------------

def run_demo():
    cap = cv2.VideoCapture(0)  # default laptop webcam

    if not cap.isOpened():
        print("ERROR: Could not open webcam.")
        return

    print("Starting ASL CPU demo. Press 'q' to quit.")
    frame_count = 0
    avg_latency_ms = 0.0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Webcam read failed.")
            break

        frame_h, frame_w = frame.shape[:2]

        # --- Simple hand ROI: central square region ---
        box_size = min(frame_w, frame_h) // 2
        x1 = frame_w // 2 - box_size // 2
        y1 = frame_h // 2 - box_size // 2
        x2 = x1 + box_size
        y2 = y1 + box_size

        roi = frame[y1:y2, x1:x2]
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Classify the ROI using full CPU model
        pred_label, pred_prob, latency_ms = classify_roi(gray_roi)

        # Running average latency for display
        frame_count += 1
        avg_latency_ms = ((avg_latency_ms * (frame_count - 1)) + latency_ms) / frame_count
        fps = 1000.0 / avg_latency_ms if avg_latency_ms > 0 else 0.0

        # --- Visual overlays ---

        # ROI box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Simple finger/hand highlighting using an Otsu mask
        _, mask = cv2.threshold(gray_roi, 0, 255,
                                cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        mask_colored = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        mask_resized = cv2.resize(mask_colored, (box_size, box_size),
                                  interpolation=cv2.INTER_NEAREST)
        overlay_region = frame[y1:y2, x1:x2]
        blended = cv2.addWeighted(overlay_region, 0.7, mask_resized, 0.3, 0)
        frame[y1:y2, x1:x2] = blended

        # Prediction text
        text = f"Pred: {pred_label} ({pred_prob*100:.1f}%), {latency_ms:.1f} ms"
        cv2.putText(frame, text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

        # Average latency / FPS
        stats = f"Avg latency: {avg_latency_ms:.1f} ms  ({fps:.1f} FPS)"
        cv2.putText(frame, stats, (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        cv2.imshow("ASL CPU Demo", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    run_demo()
